# EVALUATION

In [ ]:
from langchain.schema import format_document
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
import sys
sys.path.append('../')
from scripts.test import process_file_and_generate_vectorstore
retriever = process_file_and_generate_vectorstore("file_path")
template = """<human>: As a legal contract advisor, your role is pivotal in navigating the complexities of the contract clause agreement. Your responsibility extends beyond mere interpretation; it encompasses a meticulous analysis and strategic elucidation of the user's query based on the given {context}. To accomplish this effectively, consider the following steps:

1. Contextual Understanding: Begin by immersing yourself in the provided contract clause or {context}, grasping the intricacies of the terms, conditions, and potential implications involved. Pay particular attention to any ambiguous language or clauses that may pose risks to your client.

2. User Query Analysis: Carefully dissect the user's question within the {context} or the contract clause. Identify any specific clauses or provisions that the question pertains to and analyze their impact on your client's interests or question.

3. Comprehensive Response: Craft a response that addresses the user's query comprehensively, taking into account the nuances of the contract clause. Provide detailed explanations and recommendations, ensuring that your client is fully informed of their rights and obligations under the agreement.

4. Risk Mitigation: As you formulate your response, prioritize the mitigation of risks inherent in the contract clause. Identify potential pitfalls or areas of concern and propose proactive measures to safeguard your client's interests and minimize their exposure to liability.

5. Strategic Communication: Ensure that your response is conveyed in a clear, concise, and persuasive manner. Articulate your points effectively, citing relevant legal precedents, industry standards, and best practices to support your arguments and strengthen your client's position.

By following these steps diligently, you can provide informed, strategic guidance that empowers your client and ensures the integrity of the contract clause agreement.

NB: If the provided {context} is not sufficient enough to answer the user question please avoid any assumption.
Question: {question}


\n

<bot>:
"""


In [ ]:
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | ChatOpenAI()
    | StrOutputParser() 
)

In [ ]:
from datasets import Dataset
import time
questions = ["Who are the parties to the Agreement and what are their defined names?", 
             "What is the termination notice?",
             "What are the payments to the Advisor under the Agreement? ","Can the Agreement or any of its obligations be assigned?","Who owns the IP? ","Is there a non-compete obligation to the Advisor?"," Can the Advisor charge for meal time?","In which street does the Advisor live?","Is the Advisor entitled to social benefits?","What happens if the Advisor claims compensation based on employment relationship with the Company?"
            ]
ground_truths = [[" Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"],
                ["According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement."],
                ["According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance."],
                ["1. Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9  the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking."],
                ["According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company."],
                ["Yes. During the term of engagement with the Company and for a period of 12 months thereafter."],
                ["No. See Section 6.1, Billable Hour doesn’t include meals or travel time. "],
                ["1 Rabin st, Tel Aviv, Israel "],
                ["No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company."],
                ["If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist."]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in compression_retriever.get_relevant_documents(query)])
  time.sleep(5)
# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:

#save the files locally

output_file = "robinson_evaluation.json"
dataset.save_to_disk(output_file)

print(f"Dataset saved to {output_file}")


In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

In [ ]:
df.to_csv("evaluation_robinson_ranked.csv")

In [ ]:
import pandas as pd
df = pd.read_csv("evaluation_robinson_ranked.csv")
df.head(5)

In [ ]:
import matplotlib.pyplot as plt

# Plot each metric
plt.figure(figsize=(10, 6))
for column in ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']:
    plt.plot(df.index, df[column], marker='o', label=column)

# Add labels and legend
plt.title('Evaluation Results')
plt.xlabel('Index')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()
